LIBS

In [18]:
import pandas as pd
from  sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from string import punctuation
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
import spacy
from gensim.models.callbacks import CallbackAny2Vec

import logging
logging.basicConfig(format="%(asctime)s : - %(message)s", level=logging.INFO)

LOAD CSV

In [2]:
train_df = pd.read_csv('C:/Users/Usuario/Desktop/word_embeeding_data/treino.csv')
test_df = pd.read_csv('C:/Users/Usuario/Desktop/word_embeeding_data/teste.csv')

print(train_df.head())
print(test_df.head())

                                               title  \
0  Após polêmica, Marine Le Pen diz que abomina n...   
1  Macron e Le Pen vão ao 2º turno na França, em ...   
2  Apesar de larga vitória nas legislativas, Macr...   
3  Governo antecipa balanço, e Alckmin anuncia qu...   
4  Após queda em maio, a atividade econômica sobe...   

                                                text        date   category  \
0  A candidata da direita nacionalista à Presidên...  2017-04-28      mundo   
1  O centrista independente Emmanuel Macron e a d...  2017-04-23      mundo   
2  As eleições legislativas deste domingo (19) na...  2017-06-19      mundo   
3  O número de ocorrências de homicídios dolosos ...  2015-07-24  cotidiano   
4  A economia cresceu 0,25% no segundo trimestre,...  2017-08-17    mercado   

  subcategory                                               link  
0         NaN  http://www1.folha.uol.com.br/mundo/2017/04/187...  
1         NaN  http://www1.folha.uol.com.br/mundo/2017

SPACY
https://spacy.io/

In [3]:
nlp = spacy.load('pt_core_news_sm')

In [4]:
# txt = 'Rio de Janeiro'
# doc = nlp(txt)
# print(doc[2])
# print(doc[1].is_stop)

TRATAMENTO

In [5]:
def data_treatment(nlp, data, labels = None):
    tokens_validos = []
    labels_list = []
    step = max(1, len(data) // 50)  # Garante que step nunca seja 0
    for i, doc in enumerate(nlp.pipe(data, batch_size=1000, n_process=-1)):
        if i % step == 0:
            print(f"data {i}/{len(data)}")
        tokens = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha]
        if len(tokens) > 2:  # Adiciona apenas se houver mais de 2 palavras
            tokens_validos.append(' '.join(tokens))
            if labels is not None: labels_list.append(labels[i])

    return tokens_validos, labels_list

# treat,_ = data_treatment(nlp, ['Rio de 232 Janeiro é uma cidade','cidade cidade cidade'])

In [6]:
train_df_treat=pd.DataFrame()

train_df_treat['title'], train_df_treat['category'] = data_treatment(nlp, train_df['title'], train_df['category'])

print(train_df_treat.head())

data 0/90000
data 1800/90000
data 3600/90000
data 5400/90000
data 7200/90000
data 9000/90000
data 10800/90000
data 12600/90000
data 14400/90000
data 16200/90000
data 18000/90000
data 19800/90000
data 21600/90000
data 23400/90000
data 25200/90000
data 27000/90000
data 28800/90000
data 30600/90000
data 32400/90000
data 34200/90000
data 36000/90000
data 37800/90000
data 39600/90000
data 41400/90000
data 43200/90000
data 45000/90000
data 46800/90000
data 48600/90000
data 50400/90000
data 52200/90000
data 54000/90000
data 55800/90000
data 57600/90000
data 59400/90000
data 61200/90000
data 63000/90000
data 64800/90000
data 66600/90000
data 68400/90000
data 70200/90000
data 72000/90000
data 73800/90000
data 75600/90000
data 77400/90000
data 79200/90000
data 81000/90000
data 82800/90000
data 84600/90000
data 86400/90000
data 88200/90000
                                               title   category
0  polêmica marine le pen abomina negacionistas h...      mundo
1  macron le pen turno frança r

In [7]:
test_df_treat=pd.DataFrame()

test_df_treat['title'], test_df_treat['category'] = data_treatment(nlp, test_df['title'], test_df['category'])

print(test_df_treat.head())

data 0/20513
data 410/20513
data 820/20513
data 1230/20513
data 1640/20513
data 2050/20513
data 2460/20513
data 2870/20513
data 3280/20513
data 3690/20513
data 4100/20513
data 4510/20513
data 4920/20513
data 5330/20513
data 5740/20513
data 6150/20513
data 6560/20513
data 6970/20513
data 7380/20513
data 7790/20513
data 8200/20513
data 8610/20513
data 9020/20513
data 9430/20513
data 9840/20513
data 10250/20513
data 10660/20513
data 11070/20513
data 11480/20513
data 11890/20513
data 12300/20513
data 12710/20513
data 13120/20513
data 13530/20513
data 13940/20513
data 14350/20513
data 14760/20513
data 15170/20513
data 15580/20513
data 15990/20513
data 16400/20513
data 16810/20513
data 17220/20513
data 17630/20513
data 18040/20513
data 18450/20513
data 18860/20513
data 19270/20513
data 19680/20513
data 20090/20513
data 20500/20513
                                               title   category
0  haddad congela orçamento suspende emendas vere...    colunas
1  proposta reforma fifa divulgação

MODELO

In [8]:
'''
sg=0 -> cbow
sg=1 -> skipgram

window -> contexto das palavras(qtd antes e depois)

vector_size -> tamanho da matriz

min_count -> evita erros de digitação com um minimo necessário de repetição

alpha -> learning rate
min_alpha -> suavização do lr
'''

class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss após a época {}: {}'.format(self.epoch, loss))
        else:
            print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

model = Word2Vec(
    sg = 0,
    window=1,
    vector_size=300,
    min_count=3,
    alpha=0.03,
    min_alpha=0.007,
    compute_loss = True,
    callbacks=[callback()]
    )

token_list = [txt.split(' ') for txt in train_df_treat['title'].drop_duplicates().dropna()]
model.build_vocab(token_list, progress_per=10000)

2025-01-04 18:23:32,078 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2025-01-04T18:23:32.077543', 'gensim': '4.3.3', 'python': '3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2025-01-04 18:23:32,757 : - collecting all words and their counts
2025-01-04 18:23:32,758 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-01-04 18:23:32,796 : - PROGRESS: at sentence #10000, processed 63841 words, keeping 14987 word types
2025-01-04 18:23:32,826 : - PROGRESS: at sentence #20000, processed 127676 words, keeping 21031 word types
2025-01-04 18:23:32,865 : - PROGRESS: at sentence #30000, processed 191537 words, keeping 25492 word types
2025-01-04 18:23:32,897 : - PROGRESS: at sentence #40000, processed 255264 words, keeping 29051 word types
2025-01-04 18:23:32,934 : - PROGRESS: at sentence #50000, processed 319240 words, kee

In [9]:
model.train(
    token_list, 
    total_examples=model.corpus_count,
    epochs=30
    )

2025-01-04 18:23:33,754 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 18184 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=1 shrink_windows=True', 'datetime': '2025-01-04T18:23:33.754963', 'gensim': '4.3.3', 'python': '3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
2025-01-04 18:23:34,852 : - EPOCH 0 - PROGRESS: at 64.84% examples, 309171 words/s, in_qsize 6, out_qsize 0
2025-01-04 18:23:35,467 : - EPOCH 0: training on 540186 raw words (504151 effective words) took 1.7s, 301665 effective words/s
2025-01-04 18:23:36,530 : - EPOCH 1 - PROGRESS: at 62.99% examples, 304050 words/s, in_qsize 6, out_qsize 0
2025-01-04 18:23:37,040 : - EPOCH 1: training on 540186 raw words (504359 effective words) took 1.6s, 324804 effective words/s
2025-01-04 18:23:38,135 : - EPOCH 2 - PROGRESS: at 72.21% examples, 354171 words/s, in_qsize 6, out_qsize 0
2025-01-0

(15127970, 16205580)

In [19]:
model.wv.most_similar('google')

[('walmart', 0.5452494025230408),
 ('tesla', 0.506166934967041),
 ('autópsia', 0.47925975918769836),
 ('folhainvest', 0.4775085151195526),
 ('amazon', 0.4739232063293457),
 ('toyota', 0.46855252981185913),
 ('samsung', 0.46721357107162476),
 ('sony', 0.4558829963207245),
 ('sabmiller', 0.452098548412323),
 ('fbi', 0.44993138313293457)]

In [20]:
model.wv.save_word2vec_format('model/cbow_model.txt', binary=False)

2025-01-04 18:41:57,640 : - storing 18184x300 projection weights into model/cbow_model.txt
